<a href="https://colab.research.google.com/github/Athugodage/Hyperskill_transliteration/blob/main/transliteration_multiT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub
!pip install transformers datasets evaluation sacrebleu Sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 9.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 46.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset

In [ ]:



dataset = load_dataset('csv', 
                       data_files={'train': ['train.csv'],
                                   'test': ['test.csv']},
                       encoding='utf8',
                       index_col = False,
                       header=None,
                       sep=','
                      )

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-866682cf83f00415/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset.push_to_hub('marcus2000/polish_names')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset = load_dataset('marcus2000/polish_names')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/64 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/572 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/marcus2000___parquet/marcus2000--polish_names-fa6547915a779d8a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from transformers import AutoTokenizer, AutoModel, AutoConfig


tokenizer = AutoTokenizer.from_pretrained("t5-small", use_fast=True)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [6]:

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_hf = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['0', '1'],
        num_rows: 637
    })
})

In [7]:
max_input_length = 10
max_target_length = 10


def preprocess_function(examples):
    inputs = [ex for ex in examples['0']]  # 0 - Текст
    targets = [ex for ex in examples['1']]  # 1 - Комментарий
    model_inputs = tokenizer(inputs, 
                             max_length=max_input_length,
                             truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
     

In [8]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
     

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/572 [00:00<?, ? examples/s]

In [21]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 1

args = Seq2SeqTrainingArguments("polish_transliterator_T5",
    evaluation_strategy = "epoch",
    learning_rate=3e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    push_to_hub=True,
    predict_with_generate=True,
    fp16=True
)
    

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_hf)

In [14]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=97161131a97148f212bea2dbc12e42c6204d4c27faa4fd392b54ce220efb4c86
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [15]:
import nltk
import numpy as np
from datasets import load_metric

nltk.download("punkt", quiet=True)

metric = load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [23]:
trainer = Seq2SeqTrainer(
    model_hf,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

Cloning https://huggingface.co/marcus2000/polish_transliterator_T5 into local empty directory.


In [24]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.024200,1.807576,3.593700,0.000000,3.750000,3.750000,1.250000
2,2.829600,1.699696,4.687500,0.000000,4.687500,4.687500,0.703100
3,2.470700,1.571682,6.041700,0.000000,6.250000,6.354200,1.171900
4,2.436700,1.461692,6.406200,0.000000,6.875000,6.875000,0.968800
5,2.296000,1.384682,8.437500,0.000000,8.125000,8.437500,1.390600
6,2.090500,1.317703,8.437500,0.000000,8.125000,8.437500,1.968800
7,1.822300,1.264453,9.375000,0.000000,9.375000,9.375000,2.312500
8,1.688100,1.215687,10.625000,0.000000,10.625000,10.937500,2.796900
9,1.665500,1.184053,12.500000,0.000000,12.291700,12.500000,3.156200
10,1.573600,1.158167,13.489600,0.000000,13.333300,13.333300,3.250000


TrainOutput(global_step=11440, training_loss=1.7379568726866395, metrics={'train_runtime': 1103.1864, 'train_samples_per_second': 10.37, 'train_steps_per_second': 10.37, 'total_flos': 30240433766400.0, 'train_loss': 1.7379568726866395, 'epoch': 20.0})

In [25]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file runs/Apr14_12-12-41_aba63c29e0b9/events.out.tfevents.1681474374.aba63c29e0b9.224.4:   0%|         …

To https://huggingface.co/marcus2000/polish_transliterator_T5
   2dd617f..0a70d50  main -> main

   2dd617f..0a70d50  main -> main

To https://huggingface.co/marcus2000/polish_transliterator_T5
   0a70d50..8cbef0c  main -> main

   0a70d50..8cbef0c  main -> main



'https://huggingface.co/marcus2000/polish_transliterator_T5/commit/0a70d502f1979d09b0f41f25b353311361b0d58b'

In [26]:
from transformers import pipeline

model = pipeline('text2text-generation', model='marcus2000/polish_transliterator_T5')

In [27]:
for name in dataset['test']['0']:
  print('Polish: ', name, 'English: ', model(name)[0]['generated_text'])

Polish:  Żelisław English:  elisaw
Polish:  Wyspy Salomona English:  Wyspy Salomona
Polish:  Ginter English:  
Polish:  Karina English:  Karina
Polish:  Leo English:  Leo
Polish:  Słowacja English:  Sowacja
Polish:  Dobrosław English:  Dobrosaw
Polish:  Halina English:  Halina
Polish:  Nowa Zelandia English:  
Polish:  Romuald English:  Romuald
Polish:  Liban English:  Liban
Polish:  Witold English:  
Polish:  Izrael English:  Izrael
Polish:  Słowenia English:  Sowenia
Polish:  Korea Północna English:  Korea
Polish:  Somalia English:  Somalia
Polish:  Gwinea English:  
Polish:  Albert English:  Albert
Polish:  Aleksandra English:  Aleksandra
Polish:  Rozalia English:  Rozalia
Polish:  Turcja English:  Turcja
Polish:  Bogusław English:  Bogus
Polish:  Maria English:  Maria
Polish:  Angela English:  Angela
Polish:  Antonina English:  Antonina
Polish:  Renata English:  Renata
Polish:  Janusz English:  Janusz
Polish:  Egipt English:  Egipt
Polish:  Ireneusz English:  Ireneusz
Polish:  Mari

In [ ]:
model('Aleksander')

[{'generated_text': 'Aleksander'}]